## Imports

In [1]:
import json
import numpy as np
import re
import random

import spacy
space = spacy.load('en_core_web_sm')

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

from keras.layers import Input,Embedding,Bidirectional,LSTM,Dense,Concatenate
from keras.models import Model

## Data Preprocessing

In [2]:
with open('Intent.json') as f:
    intents = json.load(f)

In [3]:
def clean_text(text):
    text = re.sub(r'[^a-zA-z]', ' ', text)
    text = re.sub(r'[ ]+', ' ', text)
    text = text.strip()
    text = text.lower()
    return text

In [4]:
inputs = []
targets = []
classes = []
intent_dict = {}

In [5]:
for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
        
    if intent['intent'] not in intent_dict:
        intent_dict[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(clean_text(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_dict[intent['intent']].append(response)

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='*UNKNOWN*')
tokenizer.fit_on_texts(inputs)
seq = tokenizer.texts_to_sequences(inputs)
seq = tf.keras.preprocessing.sequence.pad_sequences(seq, padding='pre')

In [7]:
target_dict = {}
cat_target = []
i = -1
for target in targets:
    if target not in target_dict.values():
        target_dict[i+1] = target
        i+=1   
    cat_target.append(i)
    
cat_tensor = tf.keras.utils.to_categorical(cat_target, num_classes=len(target_dict), dtype='int32')

In [8]:
x = seq
y = cat_tensor

## Model

In [9]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 512))
model.add(Bidirectional(tf.keras.layers.LSTM(128, dropout=0.5)))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 512)         60928     
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               656384    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 22)                2838      
Total params: 753,046
Trainable params: 753,046
Non-trainable params: 0
_________________________________________________________________


In [10]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
model.fit(x, y, epochs=100, batch_size=5, verbose=1, callbacks=[early_stop])

Epoch 1/100
29/29 [==============================] - 3s 14ms/step - loss: 3.0856 - accuracy: 0.1094
Epoch 2/100
29/29 [==============================] - 0s 15ms/step - loss: 2.9736 - accuracy: 0.2073
Epoch 3/100
29/29 [==============================] - 0s 16ms/step - loss: 2.5331 - accuracy: 0.3533
Epoch 4/100
29/29 [==============================] - 0s 14ms/step - loss: 1.6944 - accuracy: 0.4844
Epoch 5/100
29/29 [==============================] - 0s 15ms/step - loss: 1.0457 - accuracy: 0.6434
Epoch 6/100
29/29 [==============================] - 0s 14ms/step - loss: 0.5304 - accuracy: 0.8239
Epoch 7/100
29/29 [==============================] - 0s 13ms/step - loss: 0.4535 - accuracy: 0.8353
Epoch 8/100
29/29 [==============================] - 0s 14ms/step - loss: 0.3224 - accuracy: 0.9343
Epoch 9/100
29/29 [==============================] - 0s 13ms/step - loss: 0.3259 - accuracy: 0.8968
Epoch 10/100
29/29 [==============================] - 0s 14ms/step - loss: 0.1247 - accuracy: 0.9958

### Demo

In [11]:
def module(text):
    seq = []
    
    text = clean_text(text)
    text = space(repr(text))
    
    for word in text:
        if word.text in tokenizer.word_index:
            seq.append(tokenizer.word_index[word.text])
        else:
            seq.append(tokenizer.word_index['*UNKNOWN*'])
            
    seq = tf.expand_dims(seq, 0)
    pred = model(seq)
    pred_class = np.argmax(pred.numpy(), axis=1)
    
    return random.choice(intent_dict[target_dict[pred_class[0]]]), target_dict[pred_class[0]]

In [14]:
print("Type 'quit' to close the chat")
while True:
    text = input('You: ')
    if text.lower() == 'quit':
        break
    response, class_pred = module(text)
    print('ChatBot: {} \n'.format(response))

Type 'quit' to close the chat
You: Hello
ChatBot: Hi human, please tell me your GeniSys user 

You: How are you?
ChatBot: Hi, how are you? I am great thanks! Please tell me your GeniSys user 

You: I am good, thank you!
ChatBot: OK! Hola <HUMAN>, how can I help you? 

You: what is your name?
ChatBot: My real name is GeniSys 

You: are you alive?
ChatBot: Thanks, I was trained that way 

You: quit
